<a href="https://colab.research.google.com/github/mequanent/Bioinformatics-hw2/blob/main/hws_110761504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the repository first using the following command
!npx degit https://github.com/mequanent/Bioinformatics-hw2 -f

npx: installed 1 in 2.226s
> destination directory is not empty. Using --force, continuing
> cloned mequanent/Bioinformatics-hw2#HEAD


**Homework 2**

In [3]:
#pip install Bio   # Please uncomment for the .py format

In [ ]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True) # Suppress displaying the scientific notation

#To install the Bio library into my IDLE python editor I used the following two steps.
#    1. cd C:\Users\USER\AppData\Local\Programs\Python\Python39\Scripts
#    2. pip install Bio
#pip install Bio

# Read fasta and parse the seq_names and the sequences
# Reference for parsing the fasta is the original biopython library documentation
# (https://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc11 section 2.4.1)

from Bio import SeqIO
seq_name = []
sequence = []
# Remember to customize as --input from CLI args
for seq_record in SeqIO.parse("test.fasta", "fasta"):
  seq_name.append(seq_record.id)
  #sequence.append(repr(seq_record.seq))
  sequence.append(seq_record.seq)
  #print(len(seq_record))
print(sequence[1])
print(sequence[2])

def preprocess_score(score):
  file = open(score, 'r')
  lines = file.readlines()#[10:] # The first 10 lines are strings
  file.close()
  columns = []
  mut = []
  for line in lines:
    if line.startswith("#") or line.startswith(" "):
      continue
    columns.append(line[0]) # Amino acid alphabets stored for row & column indexing
    line = line[1:].strip().split() # remove the letters from left and the spaces
    mut.append(line)
  mut = np.array(mut).astype(int) # change the string numbers into type int
  print(mut.shape)
  size = len(mut) # save the size of the array for future usages to come below
  return pd.DataFrame(mut, index = columns, columns = columns)

score = "pam250.txt" # score from command line args
score = preprocess_score(score) # score made a data frame to access its values with 
                  # alphabet indices
print(score)

gop = -10
gex = -2
# A function to determine pairwise alignment score
def pairwise(x, y):
  pair_sum = 0
  x_prev = '' # x_prev and y_prev are used to store immediate previous alphabets
  y_prev = '' # so that extension gaps will be identified
  for i in range(len(x)): # Considering that both x and y have the same length
    x_prev = x_prev == '-' and x[i] == '-' # status of extension gap in the first paired sequence
    y_prev = y_prev == '-' and y[i] == '-' # status of extension gap in second paired sequence
    curr = x[i] == '-' or y[i] == '-' # status of open gap in the paired sequence
    if x_prev or y_prev: # extension gap value will be taken if holds
      pair_sum = pair_sum + gex
    elif curr: # open gap value will be taken if holds
      pair_sum = pair_sum + gop
    else: # score matrix value will be taken otherwise
      pair_sum = pair_sum + score.loc[x[i]][y[i]] 
    x_prev = x[i]
    y_prev = y[i]
  return pair_sum

pairwise(sequence[0], sequence[1])

# A function to find the total sum of pairs score.
def sumOfPairs(seq):
  sop = 0 # to store sum of pairs value
  for i in range(len(seq)-1): # i iterates as long as there is j in seq as its next sequence
    for j in range(i+1, len(seq)): # j will be the next seq index to be compared with seq[i]
      sop = sop + pairwise(seq[i], seq[j])
  return sop
print(sumOfPairs(sequence))

KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEI--WWARLNDKEGYVPRNLLGLYP
(24, 24)
   A  R  N  D   C  Q  E  G  H  I  ...  P  S  T   W   Y  V  B  Z  X  *
A  2 -2  0  0  -2  0  0  1 -1 -1  ...  1  1  1  -6  -3  0  0  0  0 -8
R -2  6  0 -1  -4  1 -1 -3  2 -2  ...  0  0 -1   2  -4 -2 -1  0 -1 -8
N  0  0  2  2  -4  1  1  0  2 -2  ...  0  1  0  -4  -2 -2  2  1  0 -8
D  0 -1  2  4  -5  2  3  1  1 -2  ... -1  0  0  -7  -4 -2  3  3 -1 -8
C -2 -4 -4 -5  12 -5 -5 -3 -3 -2  ... -3  0 -2  -8   0 -2 -4 -5 -3 -8
Q  0  1  1  2  -5  4  2 -1  3 -2  ...  0 -1 -1  -5  -4 -2  1  3 -1 -8
E  0 -1  1  3  -5  2  4  0  1 -2  ... -1  0  0  -7  -4 -2  3  3 -1 -8
G  1 -3  0  1  -3 -1  0  5 -2 -3  ...  0  1  0  -7  -5 -1  0  0 -1 -8
H -1  2  2  1  -3  3  1 -2  6 -2  ...  0 -1 -1  -3   0 -2  1  2 -1 -8
I -1 -2 -2 -2  -2 -2 -2 -3 -2  5  ... -2 -1  0  -5  -1  4 -2 -2 -1 -8
L -2 -3 -3 -4  -6 -2 -3 -4 -2  2  ... -3 -3 -2  -2  -1  2 -3 -3 -1 -8
K -1  3  1  0  -5  1  0 -2  0

**Homework 3**

In [5]:
import numpy as np
import pandas as pd

from Bio import SeqIO
seq_name = []
sequence = []
# Remember to customize as --input from CLI args
for seq_record in SeqIO.parse("test3.fasta", "fasta"):
  seq_name.append(seq_record.id)
  sequence.append(seq_record.seq)
print("----- Given Sequences ---------")
print(sequence[0], len(sequence[0]))
print(sequence[1], len(sequence[1]))

def preprocess_score(score):
  file = open(score, 'r')
  lines = file.readlines()#[10:] # The first 10 lines are strings
  file.close()
  columns = []
  mut = []
  for line in lines:
    if line.startswith("#") or line.startswith(" "):
      continue
    columns.append(line[0]) # Amino acid alphabets stored for row & column indexing
    line = line[1:].strip().split() # remove the letters from left and the spaces
    mut.append(line)
  mut = np.array(mut).astype(int) # change the string numbers into type int
  return pd.DataFrame(mut, index = columns, columns = columns)

score = "pam250.txt" # score from command line args
score = preprocess_score(score) # score made a data frame to access its values with 
                  # alphabet indices
print("----- PAM Score Matrix ---------")
#print(score)

----- Given Sequences ---------
NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVN 57
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP 60
----- PAM Score Matrix ---------


In [25]:
seq1 = sequence[0]
seq2 = sequence[1]
gap = -10
# Find score_m
score_m = np.zeros((len(seq2) + 1, len(seq1) + 1))
for i in range(len(seq1) + 1):
  score_m[0][i] = i * gap
for j in range(len(seq2) + 1):
  score_m[j][0] = j * gap#score.loc[seq1[i]][seq2[j]]
score_m

array([[   0.,  -10.,  -20., ..., -550., -560., -570.],
       [ -10.,    0.,    0., ...,    0.,    0.,    0.],
       [ -20.,    0.,    0., ...,    0.,    0.,    0.],
       ...,
       [-580.,    0.,    0., ...,    0.,    0.,    0.],
       [-590.,    0.,    0., ...,    0.,    0.,    0.],
       [-600.,    0.,    0., ...,    0.,    0.,    0.]])

In [27]:
TOP = gap
DIAGONAL = 0
LEFT = gap
col = '-' + seq1
row = '-' + seq2
F = pd.DataFrame(score_m, index = row, columns = col)
F

,-,N,L,F,V,A,L,Y,D,F,...,V,P,S,N,Y,I,T,P,V,N
-,0.0,-10.0,-20.0,-30.0,-40.0,-50.0,-60.0,-70.0,-80.0,-90.0,...,-480.0,-490.0,-500.0,-510.0,-520.0,-530.0,-540.0,-550.0,-560.0,-570.0
K,-10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,-20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
V,-30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I,-40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L,-560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,-570.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L,-580.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Y,-590.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
print(col)
print(row)

-NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVN
-KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP


In [37]:
score.loc['K']['N']


1

In [36]:
for i in range(1, len(row)):
  for j in range(1, len(col)):
    TOP = score_m[i,j-1] + gap
    DIAGONAL = score_m[i-1][j-1] + score.loc[row[i]][col[j]]
    LEFT = score_m[i-1,j] + gap
    score_m[i][j] = max(TOP, DIAGONAL, LEFT) 
score_m

array([[   0.,  -10.,  -20., ..., -550., -560., -570.],
       [ -10.,    1.,   -9., ..., -535., -545., -555.],
       [ -20.,   -9.,   -3., ..., -520., -530., -540.],
       ...,
       [-580., -568., -552., ...,   40.,   35.,   25.],
       [-590., -578., -562., ...,   30.,   38.,   33.],
       [-600., -588., -572., ...,   29.,   29.,   38.]])

In [ ]:
align = np.zeros((len(seq1) + 1, len(seq2) + 1))

for i in range(len(seq1)):
  align[]
for j in range(len(seq2)):
    if seq1[] == seq2[j]: